In [1]:
import numpy as np
import pandas as pd
import os
import glob
import sklearn
from sklearn.decomposition import PCA, NMF

In [2]:
PATH = "database/Laboratory/"

In [3]:
directories = glob.glob(f"{PATH}*.csv")

In [4]:
directories

['database/Laboratory\\L10AM.csv',
 'database/Laboratory\\L13+.csv',
 'database/Laboratory\\LAB10.csv',
 'database/Laboratory\\LAB11.csv',
 'database/Laboratory\\LAB13.csv',
 'database/Laboratory\\LAB16.csv',
 'database/Laboratory\\SSCARD_A.csv']

In [5]:
full_dfs = {}

In [6]:
for direc in directories:
    name = direc.split("\\")[1].split(".")[0]
    full_dfs[name] = pd.read_csv(direc, index_col=False)

In [7]:
L10AM = full_dfs["L10AM"]
L13 = full_dfs["L13+"]
LAB10 = full_dfs["LAB10"]
LAB11 = full_dfs["LAB11"]
LAB13 = full_dfs["LAB13"]
LAB16 = full_dfs["LAB16"]
SSCARD_A = full_dfs["SSCARD_A"]

In [30]:
def merge_effectively(not_merged, df):
    max_entry = -1
    max_candidate = None
    print(f"Progressing {not_merged}")
    name = None
    for cand in not_merged:
        tmp_df_i = full_dfs[cand]
        tmp_df = pd.merge(df, tmp_df_i, on="SEQN", how="inner", suffixes=["", "**"])
        if(len(tmp_df) > max_entry):
            max_entry = len(tmp_df)
            max_candidate = tmp_df
            name = cand
    not_merged.remove(name)
    return max_candidate

In [31]:
def get_largest_df():
    max_entry = -1
    max_df = None
    for n in full_dfs.keys():
        tmp_df = full_dfs[n]
        if(max_entry < len(tmp_df)):
            max_enrty = len(tmp_df)
            max_df = n
    return max_df

In [32]:
def merge_dataframes(not_merged):
    max_df = get_largest_df()
    merged_df = full_dfs[max_df]
    not_merged.remove(max_df)
    while len(not_merged) > 0:
        merged_df = merge_effectively(not_merged, merged_df)
    return merged_df

In [43]:
not_merged = list(full_dfs.keys())
merged_df = merge_dataframes(not_merged)

Progressing ['L10AM', 'L13+', 'LAB10', 'LAB11', 'LAB13', 'LAB16']
Progressing ['L10AM', 'L13+', 'LAB10', 'LAB11', 'LAB16']
Progressing ['L10AM', 'L13+', 'LAB10', 'LAB11']
Progressing ['L10AM', 'L13+', 'LAB11']
Progressing ['L13+', 'LAB11']
Progressing ['LAB11']


In [44]:
deleted_columns = []
for i in merged_df.columns:
    if i[-2]+i[-1] == "**":
        deleted_columns.append(i)
merged_df.drop(columns=deleted_columns, inplace=True)

In [45]:
merged_df.drop(columns=["year_y", "WTSAF4YR"], inplace=True)

In [47]:
merged_df.columns

Index(['SEQN', 'WTSSCB2Y', 'SSB2M', 'SSB2ML', 'SSCYST', 'SSCYSTL', 'SSGALB',
       'SSGALBL', 'SSALB', 'SSALBL', 'SSGALBP', 'SSPRIS', 'year', 'LBXTC',
       'LBDTCSI', 'LBDHDL', 'LBDHDLSI', 'URXUMA', 'URXUMASI', 'URXUCR',
       'URXUCRSI', 'LBXGH', 'WTSAF2YR', 'LBXGLU', 'LBXGLUSI', 'LBXCPSI',
       'LBXIN', 'LBXINSI', 'LBDINSI', 'LBXTR', 'LBDTRSI', 'LBDLDL', 'LBDLDLSI',
       'LBXCRP', 'LBXBAP'],
      dtype='object')